In [1]:
import math
import json
import requests
from pathlib import Path
from datetime import datetime
import numpy as np
from typing import List, Dict, Optional

import no_gap_align

# import importlib
# importlib.reload(no_gap_align)

In [2]:
DIR = Path('/home/adam/Workspace/C#/SecStrAnnot2_data/SecStrAPI/CytochromesP450-20200707-full/Pfam_HMM')
UNIPROTID_FILE = DIR/'PF00067.rp35.ids-wo_version'
CHUNK_SIZE = 100  # number of accessions requested from UniProt API in one call
API_URL = 'http://www.ebi.ac.uk/proteins/api/proteins?accession={uniprots}'
PARTIAL_DIR = DIR/'partial_taxonomy'
MISSING_FILE = DIR/'missing_uniprots.txt'
EXTRA_FILE = DIR/'extra_uniprots.txt'
MISSING_ORGANISM_FILE = DIR/'missing_organism.txt'
TOGETHER_FILE = DIR/'taxonomy.json'
ALIGNED_SEQUENCES_JSON = DIR/'sequences.json'
LOGO_RANGE = slice(87, 111)
AMINO_ACIDS = list('ACDEFGHIKLMNPQRSTVWXY-')
AMINO_INDEX = {aa: i for i, aa in enumerate(AMINO_ACIDS)}

In [3]:
def make_chunks(lst: List[object], chunk_size: int) -> List[List[object]]:
    n_chunks = math.ceil(len(lst) / chunk_size)
    chunks = [lst[i*chunk_size:(i+1)*chunk_size] for i in range(n_chunks)]
    assert sum(len(c) for c in chunks)==len(lst)
    return chunks

def add_to_file(lst: List[str], filename: Path) -> None:
    if len(lst) > 0:
        with open(filename, 'a') as w:
            for elem in lst:
                w.write(elem)
                w.write('\n')

def select_uniprots_from_taxon(uniprot_to_organism: Dict[str,dict], taxon: str, uniprots: Optional[List[str]] = None) -> List[str]:
    uniprots = uniprots or uniprot_to_organism.keys()
    return [uni for uni in uniprots if taxon in uniprot_to_organism.get(uni, {}).get('lineage', [])]

def remove_inserts(sequence: str) -> str:
    sequence = sequence.replace('.', '')
    return ''.join(c for c in sequence if c.isupper() or c=='-')

def save_fasta(sequences: Dict[str, str], filename: str, cut_range: slice = slice(None, None)) -> None:
    with open(filename, 'w') as w:
        for name, seq in sequences.items():
            w.write(f'>{name}\n{seq[cut_range]}\n\n')

def read_sequences_from_stockholm(filename: str, dot_as_dash: bool = False) -> Dict[str, str]:
    seq_lens = set()
    sequences = {}
    with open(filename) as r:
        for line in r:
            line = line.strip()
            if not line.startswith('#') and not line.startswith('//'):
                name, seq = line.split()
                uni_v, range = name.split('/')
                uni = uni_v.split('.')[0]
                if dot_as_dash:
                    seq = seq.replace('.', '-')
                seq = remove_inserts(seq)
                sequences[name] = seq
                seq_lens.add(len(seq))
    print(seq_lens)
    assert len(seq_lens) <= 1
    return sequences

In [4]:
# LOAD UNIPROTIDS AND SEQUENCES
sequences = read_sequences_from_stockholm(DIR/'PF00067.rp35')
with open(ALIGNED_SEQUENCES_JSON, 'w') as w:
    json.dump(sequences, w)
sequences_seed = read_sequences_from_stockholm(DIR/'PF00067.seed', dot_as_dash=True)

uniprots_inlc_duplicates = [name.split('/')[0].split('.')[0] for name in sequences.keys()]
uniprots = sorted(set(uniprots_inlc_duplicates))

{463}
{557}


In [ ]:
# DOWNLOAD ORGANISM DATA FOR UNIPROTIDS

# with open(UNIPROTID_FILE) as r:
#     uniprots = sorted(set(line.strip() for line in r))

uniprots_chunks = make_chunks(uniprots, CHUNK_SIZE)

Path(PARTIAL_DIR).mkdir(exist_ok=True, parents=True)
for i, chunk in enumerate(uniprots_chunks):
    # print('chunk', i)
    url = API_URL.format(uniprots=','.join(chunk))
    response = requests.get(url, headers={'Accept':'application/json'}).text
    js = json.loads(response)

    retrieved = [protein['accession'] for protein in js]
    missing = sorted(set(chunk) - set(retrieved))
    extra = sorted(set(retrieved) - set(chunk))
    missing_organism = []
    chunk_result = {}
    for protein in js:
        uni = protein['accession']
        organism = protein.get('organism', None)
        if organism is not None:
            chunk_result[uni] = organism
        else:
            missing_organism.append(uni)
    add_to_file(missing, MISSING_FILE)
    add_to_file(extra, EXTRA_FILE)
    add_to_file(missing_organism, MISSING_ORGANISM_FILE)
    with open(Path(PARTIAL_DIR, f'chunk_{i:03}.json'), 'w') as w:
        json.dump(chunk_result, w)

In [ ]:
# MERGE DOWNLOADED DATA INTO ONE FILE
if PARTIAL_DIR.is_dir():
    together = {}
    for file in sorted(PARTIAL_DIR.glob('chunk_*.json')):
        js = json.loads(file.read_text())
        together.update(js)
    with open(TOGETHER_FILE, 'w') as w:
        json.dump(together, w)

In [5]:
# LOAD PREVIOUSLY DOWNLOADED ORGANISM DATA
with open(TOGETHER_FILE) as r:
    together = json.load(r)

In [6]:
# PRINT SUPERKINGDOM DISTRIBUTION - Taking each UniProtID only once
for taxon in ['Eukaryota', 'Bacteria', 'Archaea', 'Viruses']:
    taxon_unis = select_uniprots_from_taxon(together, taxon)
    print(f'{taxon:10} {len(taxon_unis):5}')
print('-'*16)
print(f'{len(uniprots):16}')

Eukaryota  54453
Bacteria    8087
Archaea       98
Viruses       31
----------------
           63397


In [7]:
# PRINT SUPERKINGDOM DISTRIBUTION - Taking each UniProtID repeatedly, if found more times in alignment
total = 0
for taxon in ['Eukaryota', 'Bacteria', 'Archaea', 'Viruses']:
    taxon_unis = select_uniprots_from_taxon(together, taxon, uniprots_inlc_duplicates)
    total += len(taxon_unis)
    print(f'{taxon:10} {len(taxon_unis):5}')
print(f'{"Failed":10} {len(uniprots_inlc_duplicates)-total:5}')
print('-'*16)
print(f'{len(uniprots_inlc_duplicates):16}')

Eukaryota  59809
Bacteria    8834
Archaea      102
Viruses       31
Failed       814
----------------
           69590


In [9]:
# SAVE ALIGNMENTS IN FASTA (INCL. SELECTED EUKA, BACT...)
# with open(ALIGNED_SEQUENCES_FILE) as r:
#     sequences = json.load(r)
euka = set(select_uniprots_from_taxon(together, 'Eukaryota'))
bact = set(select_uniprots_from_taxon(together, 'Bacteria'))
sequences_euka = {name: seq for name, seq in sequences.items() if name.split('/')[0].split('.')[0] in euka}
sequences_bact = {name: seq for name, seq in sequences.items() if name.split('/')[0].split('.')[0] in bact}
save_fasta(sequences, DIR/'aln.fasta', cut_range=LOGO_RANGE)
save_fasta(sequences_euka, DIR/'aln-euka.fasta', cut_range=LOGO_RANGE)
save_fasta(sequences_bact, DIR/'aln-bact.fasta', cut_range=LOGO_RANGE)
save_fasta(sequences_seed, DIR/'aln-seed.fasta', cut_range=LOGO_RANGE)

In [10]:
# CREATE LOGOS
no_gap_align.run_logomaker(DIR/'aln.fasta', DIR/'logo.png', first_index=39, title='Helix C (Pfam RP35)')
no_gap_align.run_logomaker(DIR/'aln-euka.fasta', DIR/'logo-euka.png', first_index=39, title='Helix C (Pfam RP35 Eukaryota)')
no_gap_align.run_logomaker(DIR/'aln-bact.fasta', DIR/'logo-bact.png', first_index=39, title='Helix C (Pfam RP35 Bacteria)')
no_gap_align.run_logomaker(DIR/'aln-seed.fasta', DIR/'logo-seed.png', first_index=39, title='Helix C (Pfam seed)')

In [18]:
save_fasta(sequences_seed, DIR/'aln-seed.fasta', cut_range=slice(LOGO_RANGE.start+10, LOGO_RANGE.stop+10))
no_gap_align.run_logomaker(DIR/'aln-seed.fasta', DIR/'logo-seed.png', first_index=39, title='Helix C (Pfam seed)')

In [19]:
print(*sequences_seed.keys(), sep='\n')

CP27A_HUMAN/61-526
C11B2_MOUSE/42-496
C11B1_BOVIN/42-499
CP11A_ONCMY/50-507
CP11A_HUMAN/52-511
CP11A_BOVIN/52-510
CP19A_MOUSE/48-488
CP19A_CHICK/47-487
CP4F3_HUMAN/52-515
CP4F1_RAT/52-515
CP4B1_HUMAN/47-501
CP4AA_RAT/52-504
CP4A2_RAT/52-499
CP4C1_BLADI/37-502
CP4D1_DROME/34-507
CP3A6_RABIT/37-491
CP3A1_RAT/39-494
CP6B1_PAPPO/31-495
CP6A2_DROME/32-502
CP6A1_MUSDO/35-500
PID6_FUSSO/51-503
G3XMQ0_ASPNA/36-509
CP7A1_HUMAN/32-497
CP51_YEAST/57-521
CPXI_BACMB/12-406
CP17A_MOUSE/28-492
CP17A_HUMAN/28-493
CP17A_BOVIN/28-493
CP17A_ONCMY/34-500
CP17A_CHICK/33-496
CP2D1_RAT/37-497
CP2D4_RAT/37-497
CP2DE_BOVIN/37-497
CP2F1_HUMAN/31-488
CP2B1_RAT/31-488
CP2G1_RABIT/34-491
CP2A1_RAT/33-489
CP2H1_CHICK/33-488
CP2E1_HUMAN/33-489
CP2CN_RAT/34-491
CP270_RAT/30-486
CP2C3_RABIT/30-486
CP2C1_RABIT/30-487
CP2C7_RAT/30-487
CP2CC_RAT/30-487
C75A2_SOLME/37-498
C76A2_SOLME/36-500
TCMO_HELTU/34-499
C77A1_SOLME/28-497
C77A2_SOLME/43-509
